# Change Detection in Remote Sensing Data

This notebook demonstrates how to perform change detection between two remote sensing rasters using `rasterio` and `numpy` in Python. Change detection identifies areas of change (e.g., land cover, deforestation) between two time points.

## Prerequisites
- Install required libraries: `rasterio`, `numpy`, `matplotlib` (listed in `requirements.txt`).
- Two GeoTIFF files from different time points (e.g., `raster_t1.tif`, `raster_t2.tif`) with the same CRS, resolution, and dimensions. Replace file paths with your own data.

## Learning Objectives
- Compute pixel-wise differences between two rasters.
- Apply thresholding to identify significant changes.
- Visualize and save the change detection results.

In [ ]:
# Import required libraries
import rasterio
import numpy as np
import matplotlib.pyplot as plt

## Step 1: Load Raster Files

Load two rasters from different time points and ensure they have the same CRS and dimensions.

In [ ]:
# Define file paths for the two time points
raster_t1_path = 'raster_t1.tif'  # First time point
raster_t2_path = 'raster_t2.tif'  # Second time point

# Load rasters
with rasterio.open(raster_t1_path) as src_t1, rasterio.open(raster_t2_path) as src_t2:
    raster_t1 = src_t1.read(masked=True)  # Shape: (bands, height, width)
    raster_t2 = src_t2.read(masked=True)
    profile = src_t1.profile
    crs_t1 = src_t1.crs
    crs_t2 = src_t2.crs

# Check CRS and dimensions
if crs_t1 != crs_t2:
    print('Error: Rasters have different CRS. Reproject them first.')
elif raster_t1.shape != raster_t2.shape:
    print('Error: Rasters have different dimensions.')
else:
    print(f'Raster shape: {raster_t1.shape}')
    print(f'CRS: {crs_t1}')

## Step 2: Compute Pixel-Wise Difference

Calculate the difference between the two rasters for a specific band (e.g., band 1).

In [ ]:
# Select band for analysis (e.g., band 1)
band_idx = 0
t1_data = raster_t1[band_idx]
t2_data = raster_t2[band_idx]

# Compute difference
difference = t2_data - t1_data

# Visualize difference
plt.figure(figsize=(8, 8))
plt.imshow(difference, cmap='RdBu', vmin=-np.nanmax(np.abs(difference)), vmax=np.nanmax(np.abs(difference)))
plt.colorbar(label='Difference (t2 - t1)')
plt.title('Pixel-Wise Difference (Band 1)')
plt.xlabel('Column')
plt.ylabel('Row')
plt.show()

## Step 3: Apply Thresholding for Change Detection

Identify significant changes by applying a threshold to the difference image.

In [ ]:
# Define threshold for significant change (e.g., 2 standard deviations)
threshold = 2 * np.nanstd(difference)

# Create change mask (1 for change, 0 for no change)
change_mask = np.abs(difference) > threshold
change_mask = change_mask.astype(np.uint8)  # 0: no change, 1: change

# Visualize change mask
plt.figure(figsize=(8, 8))
plt.imshow(change_mask, cmap='binary')
plt.colorbar(label='Change (1) / No Change (0)')
plt.title('Change Detection Mask')
plt.xlabel('Column')
plt.ylabel('Row')
plt.show()

## Step 4: Save Change Detection Results

Save the difference image and change mask as GeoTIFF files.

In [ ]:
# Update profile for single-band output
output_profile = profile.copy()
output_profile.update(count=1, dtype=rasterio.float32)

# Save difference image
with rasterio.open('difference.tif', 'w', **output_profile) as dst:
    dst.write(difference, 1)

# Update profile for change mask
output_profile.update(dtype=rasterio.uint8, nodata=255)

# Save change mask
with rasterio.open('change_mask.tif', 'w', **output_profile) as dst:
    dst.write(change_mask, 1)

print('Difference image saved to: difference.tif')
print('Change mask saved to: change_mask.tif')

## Next Steps

- Replace `raster_t1.tif` and `raster_t2.tif` with your own raster files.
- Adjust the threshold value or use other methods (e.g., NDVI difference, clustering).
- Apply change detection to multiple bands or derived indices.
- Proceed to the next notebook (`19_anomaly_detection.ipynb`) for anomaly detection.

## Notes
- Ensure both rasters have the same CRS, resolution, and dimensions.
- Use no-data masks to exclude invalid pixels from calculations.
- See `docs/installation.md` for troubleshooting library installation.